# Constants

In [1]:
default_skill_value = 0 # value for a players skill if no player_skill is provided on player creation
skill_value_modifier = 0.05
total_of_decks = 8 # total amount of decks
player_num = 512 # total number of players in a tournament
num_rounds = 9 # total number of rounds in a tournament

# Player and Tournament Classes

In [2]:
import numpy as np
import pandas as pd
from threading import Lock

class Deck:
    def __init__(self, name):
        self.name = name
        self.matchup_spread = {}  # Dictionary to store win probabilities against other decks

    def set_matchup_win_prob(self, opponent_deck_name, win_prob):
        """Set the win probability against a specific opponent deck."""
        self.matchup_spread[opponent_deck_name] = win_prob

    def get_matchup_win_prob(self, opponent_deck_name):
        """Retrieve the win probability against a specific opponent deck."""
        return self.matchup_spread.get(opponent_deck_name, 0.5)  # Default to 0.5 if not set
    
    def __str__(self):
        return self.name

class DeckManager:
    def __init__(self):
        self.decks = {}

    def add_deck(self, deck):
        """Add a deck to the manager."""
        self.decks[deck.name] = deck

    def generate_win_probabilities(self, deck_names):
        """Generate win probabilities for decks."""
        num_decks = len(deck_names)
        for i in range(num_decks):
            for j in range(i + 1, num_decks):
                # Sample from a normal distribution with mean=0.5 and std=0.15
                win_prob = np.clip(np.random.normal(0.5, 0.15), 0, 1)
                
                self.decks[deck_names[i]].set_matchup_win_prob(deck_names[j], win_prob)
                self.decks[deck_names[j]].set_matchup_win_prob(deck_names[i], 1 - win_prob)

    def get_win_prob_matrix(self):
        """Retrieve the win probability matrix."""
        deck_names = list(self.decks.keys())
        num_decks = len(deck_names)
        matrix = np.zeros((num_decks, num_decks))
        
        for i in range(num_decks):
            for j in range(num_decks):
                matrix[i, j] = self.decks[deck_names[i]].get_matchup_win_prob(deck_names[j])
        
        return matrix

    def get_win_prob_dataframe(self):
        """Retrieve the win probability matrix as a DataFrame."""
        deck_names = list(self.decks.keys())
        num_decks = len(deck_names)
        matrix = np.zeros((num_decks, num_decks))
        
        for i in range(num_decks):
            for j in range(num_decks):
                matrix[i, j] = self.decks[deck_names[i]].get_matchup_win_prob(deck_names[j])
        
        # Convert to DataFrame for better visualization
        df = pd.DataFrame(matrix, index=deck_names, columns=deck_names)
        return df

class Player:
    def __init__(self, player_id, alias, player_skill=default_skill_value, deck=None):
        self.id = player_id
        self.alias = alias
        self.skill = player_skill
        self.deck = deck
        self.wins = 0
        self.losses = 0
        self.history = []  # List to store match results
        self.history_lock = Lock()
    
    def __reduce__(self):
        # The object's state is returned as a tuple:
        # (callable, arguments_to_callable, additional_state)
        # Lock object is not pickled, so we're not including it in the state.
        return (self.__class__, (self.id, self.alias, self.skill, self.deck), {'wins': self.wins, 'losses': self.losses, 'history': self.history})
    
    def __setstate__(self, state):
        self.wins = state.get('wins', 0)
        self.losses = state.get('losses', 0)
        self.history = state.get('history', [])
        self.history_lock = Lock()  # Initialize a new Lock object after unpickling

    def set_deck(self, deck):
        """Set the deck for the player."""
        self.deck = deck

    def get_deck(self):
        """Retrieve the player's deck."""
        return self.deck

    def record_match(self, tournament, tournament_round, opponent, result):
        with self.history_lock:
            self.history.append([tournament.name, tournament_round, str(opponent.id) + opponent.alias, result])
            
    def get_tournament_history(self, tournament):
        tournament_results = [history for history in self.history if history[0] == tournament.name]
        return tournament_results

    def get_tournament_standing(self, tournament):
        tournament_results = [history for history in self.history if history[0] == tournament.name]
        wins = 0
        losses = 0
        for match in tournament_results:
            result = match[-1]
            if result == "W":
                wins += 1
            elif result == "L":
                losses += 1
        return (wins, losses)

    def __str__(self):
        return f"[{self.id}] {self.alias} ({self.get_deck().name}) - Skill Level: {self.skill}"

    def __repr__(self):
        return f"[{self.id}] {self.alias} ({self.get_deck().name}) - Skill Level: {self.skill}"


class Tournament:
    def __init__(self, name, player_aliases, win_prob_matrix):
        self.name = name
        self.players = player_aliases
        self.win_prob_matrix = win_prob_matrix
        self.rounds_played = 0

    def __reduce__(self):
        # Assuming win_prob_matrix is a numpy array, which is pickleable.
        # players list is also assumed to be pickleable.
        return (self.__class__, (self.name, self.players, self.win_prob_matrix), {'rounds_played': self.rounds_played})

    def __setstate__(self, state):
        self.rounds_played = state.get('rounds_played', 0)

    def swiss_pairings(self):
        sorted_players = sorted(self.players, key=lambda x: x.get_tournament_standing(self)[0], reverse=True) # [0] is the wins side of the (wins, losses) tuple returned by get_tournament_standing
        pairings = []
        while sorted_players:
            p1 = sorted_players.pop(0)
            p2 = sorted_players.pop(0)
            pairings.append((p1, p2))
        return pairings

    def simulate_round(self):
        pairings = self.swiss_pairings()
        self.rounds_played += 1
        for p1, p2 in pairings:
            index_A = self.players.index(p1)
            index_B = self.players.index(p2)
            r = np.random.rand()
            if r < self.win_prob_matrix[index_A, index_B]:
                p1.record_match(self, self.rounds_played, p2, 'W')
                p2.record_match(self, self.rounds_played, p1, 'L')
            else:
                p1.record_match(self, self.rounds_played, p2, 'L')
                p2.record_match(self, self.rounds_played, p1, 'W')
        

    def simulate_tournament(self, num_rounds):
        for _ in range(num_rounds):
            self.simulate_round()

    def display_results(self):
        for player in self.players:
            print(f"{player.alias}: {player.get_tournament_history(self)}")
    
    def display_standings(self):
        for player in self.players:
            wins, losses = player.get_tournament_standing(self)
            print(f"{player}: {wins}-{losses}")
    
    def get_top_n(self, n):
        sorted_players = sorted(self.players, key=lambda player: player.get_tournament_standing(self)[0], reverse=True)
        return sorted_players[:n]
    
    def get_win_prob_dataframe(self):
        """Retrieve the win probability for each player matrix as a DataFrame."""
        player_names = [str(player.id) + player.alias for player in self.players]
        
        # Convert to DataFrame for better visualization
        df = pd.DataFrame(self.win_prob_matrix, index=player_names, columns=player_names)
        return df
    
    def __str__(self):
        return f"{self.name}"

    def __repr__(self):
        return f"{self.name}"


# Example Setup

In [4]:
from names import get_first_name
from random import sample, choice

# 1. Generate 4 decks and their matchup spreads.
deck_names = ["Bravo", "Iyslander", "Dromai", "Katsu", "Rhinar", "Fai", "Boltyn", "Azalea"]
deck_manager = DeckManager()

for name in deck_names:
    deck = Deck(name)
    deck_manager.add_deck(deck)

deck_manager.generate_win_probabilities(deck_names)

# 2. Create `player_num` players with generic names.
players = []
for i in range(player_num):
    players.append(Player(i, get_first_name()))

# 3. Sample the `skill_level` for each player.
skill_levels = np.random.normal(0, 1, player_num)
for i, player in enumerate(players):
    player.skill = skill_levels[i]

# 4. Assign a deck to each player.
available_decks = [choice(list(deck_manager.decks.values())) for _ in range(player_num)]

for i, player in enumerate(players):
    player.set_deck(available_decks[i])

# 5. Generate the `win_prob_matrix`.
tournament_win_prob_matrix = np.zeros((player_num, player_num))
for i in range(player_num):
    for j in range(player_num):
        if i != j:
            deck_win_prob = players[i].get_deck().get_matchup_win_prob(players[j].get_deck().name)
            skill_adjustment = skill_value_modifier * (players[i].skill - players[j].skill)
            tournament_win_prob_matrix[i, j] = deck_win_prob + skill_adjustment
            tournament_win_prob_matrix[j, i] = 1 - tournament_win_prob_matrix[i, j]


# Example usage

In [5]:
print("Matchup spread for each deck:")
print(deck_manager.get_win_prob_dataframe())

print('\n\nPlayers in the tournament:')
for i in range(0, player_num):
    print(players[i])

Matchup spread for each deck:
              Bravo  Iyslander    Dromai     Katsu    Rhinar       Fai  \
Bravo      0.500000   0.494067  0.593187  0.464889  0.510416  0.665559   
Iyslander  0.505933   0.500000  0.667457  0.548241  0.688812  0.477281   
Dromai     0.406813   0.332543  0.500000  0.467226  0.091635  0.639457   
Katsu      0.535111   0.451759  0.532774  0.500000  0.275651  0.371318   
Rhinar     0.489584   0.311188  0.908365  0.724349  0.500000  0.449507   
Fai        0.334441   0.522719  0.360543  0.628682  0.550493  0.500000   
Boltyn     0.487879   0.565619  0.367084  0.640990  0.615647  0.364614   
Azalea     0.716434   0.424288  0.612224  0.805467  0.484379  0.592139   

             Boltyn    Azalea  
Bravo      0.512121  0.283566  
Iyslander  0.434381  0.575712  
Dromai     0.632916  0.387776  
Katsu      0.359010  0.194533  
Rhinar     0.384353  0.515621  
Fai        0.635386  0.407861  
Boltyn     0.500000  0.670414  
Azalea     0.329586  0.500000  


Players in th

## Make a single round tournament

In [6]:
# instantiate a tournament and run a round
one_round_tournament = Tournament("One Round Tournament", players, tournament_win_prob_matrix)
one_round_tournament.simulate_tournament(1)

# print("Round Pairings and results")
# one_round_tournament.display_results()

print("\n\nTop 8 after round 1:")
one_round_tournament.get_top_n(8)



Top 8 after round 1:


[Aiko (Azalea) - Skill Level: 1.2142178656850413,
 Chastity (Azalea) - Skill Level: -0.49827447378950707,
 Robbie (Katsu) - Skill Level: 0.16653017553799956,
 Bernice (Katsu) - Skill Level: 0.7180935060471197,
 Patricia (Dromai) - Skill Level: -0.14896145044912654,
 Robert (Katsu) - Skill Level: 1.6424828356079653,
 Conrad (Dromai) - Skill Level: 1.1444073618887027,
 Jose (Azalea) - Skill Level: -1.0095556045166927]

# Running a 9 round tournament

In [7]:
# instantiate a 9 round tournament and run it
tournament_example = Tournament("9-round tournament", players, tournament_win_prob_matrix)
tournament_example.simulate_tournament(9)

# print("Round Pairings and results")
# tournament_example.display_results()

# print("\n\nStandings:")
# tournament_example.display_standings()

print(f"Top 8:")
for player in tournament_example.get_top_n(8):
    wins, losses = player.get_tournament_standing(tournament_example)
    print(f"{player}: {wins}-{losses}")

Top 8:
Ruth (Boltyn) - Skill Level: 0.4003966995629445: 9-0
Charles (Iyslander) - Skill Level: 1.2992535695425556: 8-1
Britney (Boltyn) - Skill Level: 0.6111698741330147: 8-1
Elizabeth (Dromai) - Skill Level: 0.09360488663478356: 8-1
Donald (Bravo) - Skill Level: 0.0690242493704483: 8-1
Maria (Fai) - Skill Level: -0.635660857812095: 8-1
Nancy (Dromai) - Skill Level: 1.6638180472421376: 8-1
Benjamin (Iyslander) - Skill Level: 1.0688487844369845: 8-1


# Doing analysis on a 1k Run

In [3]:
import pickle

# Load pickled list of completed tournaments
with open("completed_tournaments.pkl", "rb")as f:
    completed_tournaments = pickle.load(f)

print(f"Total tournaments: {len(completed_tournaments)}")

EOFError: Ran out of input

In [5]:
# Utility function to find the player with the most Top 8 finishes
def find_most_top_8s(tournament_list):
    """
    Find the player with the most Top 8 finishes across all tournaments.
    
    Parameters:
    tournament_list (list): A list of Tournament objects.
    
    Returns:
    str: The alias of the player with the most Top 8 finishes.
    """
    top_8_counts = {}  # Dictionary to store the number of Top 8 finishes for each player
    for tournament in tournament_list:
        top_8_players = tournament.get_top_n(8)
        for player in top_8_players:
            if player.id in top_8_counts:
                top_8_counts[player.id] += 1
            else:
                top_8_counts[player.id] = 1

    # Find the player with the most Top 8 finishes
    most_top_8s_player = max(top_8_counts, key=top_8_counts.get)
    return most_top_8s_player, top_8_counts

# Utility function to create a DataFrame with results for every player
def create_results_dataframe(tournament_list):
    """
    Create a DataFrame with the number of Top 8 finishes for each player across all tournaments.
    
    Parameters:
    tournament_list (list): A list of Tournament objects.
    
    Returns:
    DataFrame: A DataFrame showing the number of Top 8 finishes for each player.
    """
    _, top_8_counts = find_most_top_8s(tournament_list)
    df = pd.DataFrame(list(top_8_counts.items()), columns=['Player', 'Top 8 Finishes'])
    return df.sort_values('Top 8 Finishes', ascending=False)

In [13]:
completed_tournaments[0].players[88].history

[['Tournament #0', 1, 'Melissa', 'W'],
 ['Tournament #0', 2, 'Margarita', 'W'],
 ['Tournament #0', 3, 'Lesa', 'W'],
 ['Tournament #0', 4, 'Eileen', 'W'],
 ['Tournament #0', 5, 'Adela', 'L'],
 ['Tournament #0', 6, 'Annie', 'W'],
 ['Tournament #0', 7, 'Eileen', 'W'],
 ['Tournament #0', 8, 'Adela', 'L'],
 ['Tournament #0', 9, 'Geneva', 'W']]

In [6]:
# getting statistics
most_top_8s_player, _ = find_most_top_8s(completed_tournaments)
print(f"The player with the most Top 8 finishes is {most_top_8s_player}")

df = create_results_dataframe(completed_tournaments)
print("DataFrame of Top 8 finishes for each player:")
df

The player with the most Top 8 finishes is 88
DataFrame of Top 8 finishes for each player:


,Player,Top 8 Finishes
35,88,131
80,254,125
21,351,121
1,17,102
7,343,99
...,...,...
426,498,1
428,164,1
429,243,1
225,438,1
